<a href="https://colab.research.google.com/github/isavoj/DQN/blob/master/HomeWorkA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric

     |████████████████████████████████| 7.9 MB 9.0 MB/s 
     |████████████████████████████████| 3.5 MB 10.4 MB/s 


In [3]:
!pip install git+https://github.com/bonlime/pytorch-tools.git@master

  Cloning https://github.com/bonlime/pytorch-tools.git (to revision master) to /tmp/pip-req-build-6vt95ghx
  Running command git clone -q https://github.com/bonlime/pytorch-tools.git /tmp/pip-req-build-6vt95ghx
     |████████████████████████████████| 58 kB 3.0 MB/s 
  Created wheel for pytorch-tools: filename=pytorch_tools-0.3.0-py3-none-any.whl size=156583 sha256=ff289ed31fd6526b032173989c2b08bf2c1060cf114597f906b265ee2053a8ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-v4lhlz1n/wheels/68/82/a6/2c97aac95f21ea5471389d44ee000410907930588feeaa0477
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=267e586e09cac806c3d696a3102992caa050767668849ed45e557d460e8312bc
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built pytorch-tools adamp


In [4]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [ ]:
dataset = Planetoid(root="tutorial1",name= "Cora")


## From the Planetoid Cora dataset, extract the number of nodes that are in the 
* training set, 
* validation set, 
* test set

In [ ]:
print(dataset.data)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of node features:\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)


In [ ]:
data = dataset.data
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Number of training nodes: {data.test_mask.sum()}')
print(f'Number of training nodes: {data.val_mask.sum()}')

## Discuss briefly the implications for supervised versus semi-supervised learning.

Supervised learning - data in our training set are labeled. 
unsupervised learning - training data is unlabeled. 
the the usual categories of
supervised and unsupervised are not necessarily the most informative or useful
when it comes to graphs.
Semi supervised - combination of the two above. Pseudo labeling. Machine learning models try to assign similar labels to to neigbouring nodes in a graph,

It used the semi-supervised learning method to connect clusters of data based on their similarities. Semi-supervised machine learning uses both labelled and unlabeled data. Semi-supervised algorithm partially trains itself with a small set of labelled data. It then uses this trained part to identify and label the larger unlabelled dataset. 

Semi-supervised learning helps in labelling data during cluster analysis. Cluster analysis identifies similar data and groups them. This is done using the unsupervised technique. But, semi-supervised machine learning enables the labelling of unlabelled data in the cluster. 

This method helps overcome the limitations of supervised learning. Since data-labelling is an expensive task, incorporating semi-supervised technique is cost-effective and ensures good results. 



Here, we are given the ground-truth labels of only a small subset of nodes, and want to infer the labels for all the remaining nodes (transductive learning).

### Train both a standard dense network using supervised learning and network using the convolution from equation 1 using semi-supervised learning.

# Standard network

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = MLP(hidden_channels=16)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.
patience = 4
the_last_loss = 1000000

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.

      return test_acc

def validation():
      model.eval()
      out = model(data.x)
      loss = criterion(out[data.val_mask], data.y[data.val_mask])

      return loss

for epoch in range(1, 201):
    loss = train()

    # Early stopping
    the_current_loss = validation()

    if the_current_loss > the_last_loss:
        trigger_times += 1
        print('trigger times:', trigger_times)

        if trigger_times >= patience:
            print('Early stopping!\nStart to test process.')
            break

    else:
        print('trigger times: 0')
        trigger_times = 0

    the_last_loss = the_current_loss
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')


# Graph convolution
Hard to define a kernel in graph convolution do to irregular structure. Need to aggregate node features.



In [ ]:
from torch_geometric.nn import GCNConv
# Dataset.num_features. num of features per node in a graph. For every node in the graph, get all of the attributes vectores of its connected nodes and apply some aggregation function, like an average. A node might be represented as the avregae of its neighbours.
# pass the average vector through a dense neuaral network layer. = multiply it with some matrix and apply an activation function. The output of the ense layer is the new vector representation of the node. 
# This is done for every node in the graph. ( OBS DOES THIS MEAN THAT WE HAVE NUMBER OF NETWORKS = NBr oF NODES, CREATED WITH GCN CONV)
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
patience = 4
the_last_loss = 10000
trigger_times = 0

def train():

      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.

      return loss

def test():

      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.

      return test_acc


def validation():

      model.eval()
      out = model(data.x, data.edge_index)
      loss = criterion(out[data.val_mask], data.y[data.val_mask])

      return loss

for epoch in range(1, 101):

    loss = train()
    # Early stopping
    the_current_loss = validation()

    if the_current_loss > the_last_loss:
        trigger_times += 1
        print('trigger times:', trigger_times)

        if trigger_times >= patience:
            print('Early stopping!\nStart to test process.')
            break

    else:
        print('trigger times: 0')
        trigger_times = 0

    the_last_loss = the_current_loss
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

In [ ]:
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [ ]:
model.eval()

out = model(data.x, data.edge_index)
visualize(out, color=data.y)

Try to outperform the results from (1) using one of the more advanced algorithms
such as the graph attention network, from eqn. 3-4. Try to optimize performance, vary-
ing the number of heads (for GAT), different pooling layers (?), dropout, etc. (Extensive
hyperparameter tuning however is not asked for.)
It may not be that easy to improve the results from (1) much (see Cora Benchmark), the
main objective here is to show that you have tried.

# GAT

In [16]:
from torch_geometric.nn import GATConv


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  #Compute the loss solely based on the training nodes.
      loss.backward()   #Derive gradients.
      optimizer.step()  #Update parameters based on gradients.
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc


for epoch in range(1, 201):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GAT(
  (conv1): GATConv(1433, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)
WORKS 1433
WORKS
Epoch: 001, Loss: 1.9590, Val: 0.3840, Test: 0.4590
WORKS 1433
WORKS
Epoch: 002, Loss: 1.8685, Val: 0.6040, Test: 0.6510
WORKS 1433
WORKS
Epoch: 003, Loss: 1.7665, Val: 0.6720, Test: 0.7140
WORKS 1433
WORKS
Epoch: 004, Loss: 1.6698, Val: 0.7040, Test: 0.7220
WORKS 1433
WORKS
Epoch: 005, Loss: 1.5698, Val: 0.7080, Test: 0.7250
WORKS 1433
WORKS
Epoch: 006, Loss: 1.5700, Val: 0.7260, Test: 0.7420
WORKS 1433
WORKS
Epoch: 007, Loss: 1.3842, Val: 0.7400, Test: 0.7490
WORKS 1433
WORKS
Epoch: 008, Loss: 1.3283, Val: 0.7460, Test: 0.7500
WORKS 1433
WORKS
Epoch: 009, Loss: 1.3015, Val: 0.7540, Test: 0.7560
WORKS 1433
WORKS
Epoch: 010, Loss: 1.2726, Val: 0.7560, Test: 0.7610
WORKS 1433
WORKS
Epoch: 011, Loss: 1.2053, Val: 0.7600, Test: 0.7700
WORKS 1433
WORKS
Epoch: 012, Loss: 1.1320, Val: 0.7620, Test: 0.7760
WORKS 1433
WORKS
Epoch: 013, Loss: 1.0645, Val: 0.7620, Test: 0.7770
WORKS 1433
WORKS
Epoch: 

In [1]:
torch.load('graph_list_2022.pt')

NameError: ignored